In [6]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

In [ ]:
%pip install google-cloud-bigquery
%pip install pandas
%pip install db-dtypes
%pip install pandas-gbq

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from google.cloud import bigquery
import pandas as pd

from utils.import_table import import_table
from utils.export import export_table


In [ ]:
# Initialiser le client BigQuery
client = bigquery.Client.from_service_account_json('../../key.json')

In [ ]:
date = import_table(client, 'date', dataset_id='warehouse')
display(date.head())

SyntaxError: keyword argument repeated: dataset_id (2083632516.py, line 1)

In [ ]:
po = import_table(client, 'purchase_order', dataset_id="ensemble_de_donnees_public").sort_values(by='id')

# replace date in so by date_id of date 
po = po.merge(date, on='date', how='left')
po = po.drop(columns=['date'])
po = po.rename(columns={'id_y': 'date_id', 'id_x': 'id'})
display(po.head())


/Users/antoine/miniconda3/envs/AI/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,id,supplier_id,date_id
0,1,3,283
1,2,4,551
2,3,5,1021
3,4,6,681
4,5,5,873


In [ ]:
pol = import_table(client, 'purchase_order_line', dataset_id="ensemble_de_donnees_public").sort_values(by='purchase_order_id')

pol = pol.merge(po, left_on='purchase_order_id', right_on='id', how='left')

pol = pol.drop(columns=['id_y'])
spol = pol.rename(columns={'id_x': 'id'})

display(pol.head())

/Users/antoine/miniconda3/envs/AI/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,id_x,quantity,unit_price,purchase_order_id,product_id,supplier_id,date_id
0,936,651,72,1,185,3,283
1,945,815,78,1,103,3,283
2,573,997,72,1,150,3,283
3,431,447,11,1,117,3,283
4,319,973,87,2,191,4,551


In [ ]:
stores = import_table(client, 'supplier', dataset_id="ensemble_de_donnees_public")
stores = stores[['id', 'location_id']]
display(stores.head())

pol = pol.merge(stores, left_on='supplier_id', right_on='id', how='left')
display(pol.head())
pol = pol.drop(columns=['id', 'supplier_id'])
pol = pol.rename(columns={'id_x': 'id'})

display(pol.head())

/Users/antoine/miniconda3/envs/AI/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,id,location_id
0,6,12
1,11,13
2,15,16
3,12,23
4,9,28


,id_x,quantity,unit_price,purchase_order_id,product_id,supplier_id,date_id,id,location_id
0,936,651,72,1,185,3,283,3,31
1,945,815,78,1,103,3,283,3,31
2,573,997,72,1,150,3,283,3,31
3,431,447,11,1,117,3,283,3,31
4,319,973,87,2,191,4,551,4,47


,id,quantity,unit_price,purchase_order_id,product_id,date_id,location_id
0,936,651,72,1,185,283,31
1,945,815,78,1,103,283,31
2,573,997,72,1,150,283,31
3,431,447,11,1,117,283,31
4,319,973,87,2,191,551,47


In [ ]:
pol = pol.drop(columns='id')
display(pol.head())

,quantity,unit_price,purchase_order_id,product_id,date_id,location_id
0,651,72,1,185,283,31
1,815,78,1,103,283,31
2,997,72,1,150,283,31
3,447,11,1,117,283,31
4,973,87,2,191,551,47


In [ ]:
# Variables pour BigQuery
dataset_name = "warehouse" # Nom du dataset à créer/exporter
table_name = "f_purchase" # Nom de la table à créer


In [ ]:
export_table(client, pol, dataset_name, table_name)

Dataset warehouse_test déjà existant.


100%|██████████| 1/1 [00:00<00:00, 10645.44it/s]
